In [ ]:
# Python 3.12.9

import requests
import json
import matplotlib.pyplot as plt
import numpy as np

BAR_COLORS=["khaki", "firebrick", "darkorange", "forestgreen", "royalblue", "orangered", "gold", "dodgerblue", "darkblue"]
PLOT_COLORS=list(reversed(BAR_COLORS))
PLOT_MARKERS=["v", "^", "<", ">", "8", "s", "p", "*", "H"]

In [ ]:
PROJECT_ID="socksproxyclient"
MOD_LOADERS=["fabric", "quilt"]
GAME_VERSIONS=["1.16.5", "1.18.2", "1.19.3", "1.19.4", "1.20", "1.20.1", "1.20.2", "1.20.3", "1.20.4", "1.20.5", "1.20.6", "1.21", "1.21.1", "1.21.2", "1.21.3", "1.21.4", "1.21.5"]

In [ ]:
req = requests.get(f"https://api.modrinth.com/v2/project/{PROJECT_ID}/version?loaders=[{','.join(list(map(lambda v: "\"" + v + "\"", MOD_LOADERS)))}]&game_versions=[{','.join(list(map(lambda v: "\"" + v + "\"", GAME_VERSIONS)))}]")

if (req.status_code != 200):
    print(req.status_code)
    print(req.text)
    exit(1)

print(req.headers)
rsp: list = json.loads(req.text)
print(rsp)

In [ ]:
publication_hits = {}
download_hits = {}

discovered_game_versions = set()
discovered_mod_loaders = set()

for gv in GAME_VERSIONS:
    discovered_game_versions.add(gv)
    if publication_hits.get(gv) is None:
        publication_hits[gv] = {}
    if download_hits.get(gv) is None:
        download_hits[gv] = {}
    
    for ml in MOD_LOADERS:
        discovered_mod_loaders.add(ml)
        if publication_hits[gv].get(ml) is None:
            publication_hits[gv][ml] = 0
        if download_hits[gv].get(ml) is None:
            download_hits[gv][ml] = 0
    

for item in rsp:
    for gv in item["game_versions"]:
        if gv not in GAME_VERSIONS:
            continue
        for ml in item["loaders"]:
            if ml not in MOD_LOADERS:
                continue
            publication_hits[gv][ml] += 1
            download_hits[gv][ml] += item["downloads"]

print(publication_hits)
print(download_hits)

discovered_game_versions=sorted(discovered_game_versions)
discovered_mod_loaders=sorted(discovered_mod_loaders)

In [ ]:
_figures = []

_, _pillars = plt.subplots(figsize=(19.2, 10.8))
_lines = _pillars.twinx()

_bar_width = 1.0 / len(discovered_mod_loaders) - 0.02
_x_bar_positions = [np.arange(len(discovered_game_versions))]
for i in range(1, len(discovered_mod_loaders)):
    _x_bar_positions.append(_x_bar_positions[i-1] + _bar_width)

for i in range(0, len(discovered_mod_loaders)):
    _pillar = _pillars.bar(_x_bar_positions[i], [sum([download_hits[gv][ml] for ml in download_hits[gv] if ml == discovered_mod_loaders[i]]) for gv in discovered_game_versions], width=_bar_width, label=f"{discovered_mod_loaders[i].capitalize()} downloads", color=BAR_COLORS[i])
    _figures.append(_pillar)
    _pillars.bar_label(_pillar, label_type="edge", fmt="%d", rotation=-90, fontsize=12)

_pillars.set_xlabel("Minecraft versions", fontsize=16)
_pillars.set_ylabel("Downloads", fontsize=16)
_pillars.set_xticks(ticks=_x_bar_positions[0] - _bar_width / 2, labels=discovered_game_versions, fontsize=16)
_pillars.tick_params(axis="x", direction='out', labelrotation=45)
_pillars.tick_params(axis="both", labelsize=16)
_pillars.ticklabel_format(axis="y", style="plain")
_pillars.grid(ls="--", alpha=0.5, color="gray")

for i in range(0, len(discovered_mod_loaders)):
    _figures.append(_lines.plot(discovered_game_versions, [sum([publication_hits[gv][ml] for ml in publication_hits[gv] if ml == discovered_mod_loaders[i]]) for gv in discovered_game_versions], label=f"{discovered_mod_loaders[i].capitalize()} publications", marker=PLOT_MARKERS[i], ls="--", color=PLOT_COLORS[i])[0])

_lines.set_ylabel("Publications", fontsize=16)
_lines.tick_params(axis="y", labelsize=16)
_lines.ticklabel_format(axis="y", style="plain")

plt.legend(_figures, [h.get_label() for h in _figures], fontsize=16, loc="best")
plt.title(f"Mod \"{PROJECT_ID}\" on Modrinth downloads analytics", fontsize=20)
plt.tight_layout()
plt.show()
